In [23]:

%matplotlib inline
import os
import sys
import string
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from qm import*
from math import*
from random import shuffle
import random

import subprocess

In [34]:
temp = open('styr.kiss2')
line_char = temp.readlines()
#print(line_char)

l = []
entrada = []
lista_entradas = []
saida = []
lista_saidas = []

estados = []
estados = line_char[4:5]

atual = []
atual_inteiro = []
atual_convertido = []

proximo = []
proximo_inteiro = []
proximo_convertido = []

espaco_busca = []
espaco_aleatorio = []
estados_convertido = int(estados[0].split()[1])

#define o tamanho do espaço de busca basedado no valor entregue pelo arquivo

tamanho_espaco_busca = math.ceil(np.log2(estados_convertido))

entrada_input = int(line_char[1][3:])
saida_output = int(line_char[2][3:])
numero_estados = int(line_char[4][3:])

msf_pronta = []

l = line_char[5:]


for i in range(len(l)):
    entrada = l[i].split()[0]
    lista_entradas.append(entrada)
    
    atual = l[i].split()[1]
    proximo = l[i].split()[2]
    saida = l[i].split()[3]
    lista_saidas.append(saida)
    
    atual_inteiro = int(atual[2:])
    proximo_inteiro = int(proximo[2:])
    atual_convertido.append(bin(atual_inteiro)[2:].zfill(tamanho_espaco_busca))
    #atual_binario = atual_convertido[i]
    proximo_convertido.append(bin(proximo_inteiro)[2:].zfill(tamanho_espaco_busca))
    #proximo_binario = proximo_convertido[i]
    linha = str(lista_entradas[i])+str(atual_convertido[i])+' '+str(proximo_convertido[i])+str(lista_saidas[i])
    msf_pronta.append(linha)
    #print(msf_pronta[i])
    
#msf_pronta.append('.e')
comprimento_input = str(len(lista_entradas[0])+len(atual_convertido[0]))
comprimento_output = str(len(proximo_convertido[0])+len(lista_saidas[0]))
#print(comprimento_input)


d = open('melhor.pla','w')
#d.write('\n')
d.write('.i '+comprimento_input+'\n')
d.write('.o '+comprimento_output+'\n')
d.write(line_char[3])

for i in range(len(msf_pronta)):
    d.write(msf_pronta[i]+'\n')
#msf_pronta.append('.e'.rstrip())

d.close()


In [35]:
for i in range (2**tamanho_espaco_busca):
        espaco_busca.append(bin(i)[2:].zfill(tamanho_espaco_busca))
        #espaco_aleatorio.append(bin(i)[2:].zfill(tamanho_espaco_busca))

def cria_nova_msf():
    #print(tamanho_espaco_busca)
    espaco_aleatorio = espaco_busca.copy()
    shuffle(espaco_aleatorio)
    return set(espaco_aleatorio)

def correspondente(palavra1,palavra2):
    #espaco_aleatorio = list(cria_nova_msf())
    espaco_aleatorio = palavra2
    for i in range(len(espaco_busca)):
        if palavra1 == espaco_busca[i]:
            return str(espaco_aleatorio[i])
        
def nova_maquina():
    novo_atual = []
    novo_proximo = []
    nova_msf = []
    
    #esse trecho faz 5 embaralhamentos da lista para gerar valores aleatorio a cara criacao de uma conjunto de atribuicoes
    
    espaco_aleatorio = espaco_busca.copy()
    for i in range(3):
        shuffle(espaco_aleatorio)
    
    # percebi que esse trecho estava gerando uma nova lista de atribuicoes a cada chamada de teste de correspondencia
    # creio que isso estava causando um efeito indesejado, uma que vez isso provavelmente estava gerando inconsistencias 
    # que podem estar passando batidas
    # então agora é criada apenas uma lista que é utilizada para a comparacao entre atual e proximos estados da primeira 
    # lista e do novo conjunto de atribuicoes
    
    # aparentemente essa alteração, que pra mim faz sentido, fez cair muito a quantidade de geracao de msf's de tamanho menor que
    # a inicial
    
    for i in range(len(l)):
        #print(atual_convertido[i],proximo_convertido[i])
        novo_atual.append(correspondente(atual_convertido[i],espaco_aleatorio))
        #print("novo atual:",novo_atual[i])
        novo_proximo.append(correspondente(proximo_convertido[i],espaco_aleatorio))
        #print("novo proximo:",novo_proximo[i])
        nova_atrib = str(lista_entradas[i])+str(novo_atual[i])+' '+str(novo_proximo[i])+str(lista_saidas[i])
        nova_msf.append(nova_atrib)
        
    d = open('nova.pla','w')
    #d.write('\n')
    d.write('.i '+comprimento_input+'\n')
    d.write('.o '+comprimento_output+'\n')
    d.write(line_char[3])

    for i in range(len(msf_pronta)):
        d.write(nova_msf[i]+'\n')
    #msf_pronta.append('.e'.rstrip())
    
    d.close()
    
    #return nova_msf

In [36]:
def simplifica_nova_maquina():
    teste = subprocess.run(['espresso','nova.pla'],stdout=subprocess.PIPE,universal_newlines=True)
    saida = teste.stdout
    a = open('nova.pla','w')
    a.write(saida)
    a.close()
    
def simplifica_melhor_maquina():
    teste = subprocess.run(['espresso','melhor.pla'],stdout=subprocess.PIPE,universal_newlines=True)
    saida = teste.stdout
    a = open('melhor.pla','w')
    a.write(saida)
    a.close()

In [37]:
def leitura_nova_msf():
    nova = open('nova.pla','r')
    line_char = nova.readlines()
    custo_nova = int(line_char[2][3:])
    nova.close()
    return custo_nova

def leitura_melhor_msf():
    melhor = open('melhor.pla','r')
    line_char = melhor.readlines()
    custo_melhor = int(line_char[2][3:])
    melhor.close()
    return custo_melhor

In [38]:
def troca_maquina():
    nova = open('nova.pla','r')
    melhor = open('melhor.pla','w')
    nova_lista = nova.readlines()
    for i in range(len(nova_lista)):
        melhor.write(str(nova_lista[i]))

In [39]:
def calculo_custo_nova():
    transicoes_nova = leitura_nova_msf()
    area = transicoes_nova*(2*entrada_input + 3*(np.log2(numero_estados)) + saida_output)
    return area

def calculo_custo_melhor():
    transicoes_melhor = leitura_melhor_msf()
    area = transicoes_melhor*(2*entrada_input + 3*(np.log2(numero_estados)) + saida_output)
    return area


In [40]:
def resfriamento_simulado(temperatura):
    temperatura_inicial = temperatura
    temperatura_final = 10
    simplifica_melhor_maquina()
    custo_melhor = calculo_custo_melhor()
    print("melhor custo",custo_melhor)
    while(temperatura > temperatura_final):
        for i in range(2):
            custo_melhor = calculo_custo_melhor()
            nova_maquina()
            simplifica_nova_maquina()
            custo_nova = calculo_custo_nova()
            probabilidade = np.random.random_sample()
            if (custo_nova - custo_melhor <= 0) or (probabilidade < np.exp(-(custo_nova-custo_melhor)/temperatura)):
                troca_maquina()
                custo_melhor = calculo_custo_melhor()
                print("houve troca")
                
        
        print("T ->",temperatura,"custo->",custo_melhor)
        if temperatura > temperatura_inicial/2:
            temperatura = 0.8*temperatura
        else:
            temperatura = 0.5*temperatura
            
    return custo_melhor

In [45]:
temperatura = 300
resfriamento_simulado(temperatura)

melhor custo 4656.553224763986
T -> 300 custo-> 4656.553224763986
T -> 240.0 custo-> 4656.553224763986
T -> 192.0 custo-> 4656.553224763986
houve troca
T -> 153.60000000000002 custo-> 4955.597927271764
T -> 122.88000000000002 custo-> 4955.597927271764
T -> 61.44000000000001 custo-> 4955.597927271764
T -> 30.720000000000006 custo-> 4955.597927271764
T -> 15.360000000000003 custo-> 4955.597927271764


4955.597927271764